In [4]:
%pip install lxml
import os
import s3fs
import pandas as pd
import xml.etree.ElementTree as ET
from lxml import etree

os.environ["AWS_ACCESS_KEY_ID"] = 'FKZHZ6Z5IFSSHLF7MFG4'
os.environ["AWS_SECRET_ACCESS_KEY"] = 'X7PkfiBZ0BwZRu1qHBw2U3OzI53hbX1pfQiSK64i'
os.environ["AWS_SESSION_TOKEN"] = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJGS1pIWjZaNUlGU1NITEY3TUZHNCIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzYxNzI3MDI4LCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6Imx1Y2FzLmN1bXVuZWxAZW5zYWUuZnIiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzYyODc4MDk2LCJmYW1pbHlfbmFtZSI6IkN1bXVuZWwiLCJnaXZlbl9uYW1lIjoiTHVjYXMiLCJncm91cHMiOlsiVVNFUl9PTllYSUEiLCJzdGF0YXBwLXNlZ21lZGljIl0sImlhdCI6MTc2MjI3MzI5NiwiaXNzIjoiaHR0cHM6Ly9hdXRoLmxhYi5zc3BjbG91ZC5mci9hdXRoL3JlYWxtcy9zc3BjbG91ZCIsImp0aSI6Im9ucnRydDoxMzYyYjU1MS04YTFjLTEzMDctYTBlYi0zMTk0ZTY0NWUxYzEiLCJuYW1lIjoiTHVjYXMgQ3VtdW5lbCIsInBvbGljeSI6InN0c29ubHkiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJsYWIiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiIsImRlZmF1bHQtcm9sZXMtc3NwY2xvdWQiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1zc3BjbG91ZCJdLCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGdyb3VwcyBlbWFpbCIsInNpZCI6IjYwZmE3OTJjLWUxZDctNDZjYy1iNDc1LTViMzE3N2ZlYjI3MCIsInN1YiI6ImUyZDc4NjRjLTcwMzItNDI0ZC04OTA2LWU0ZjhiNDFjYzAwMyIsInR5cCI6IkJlYXJlciJ9.YkumQsf-x7Q1tMqtScQRIKYo2Gr-0hyiZroe2RH3IQ9yMOOdTOiwmEwGPRGhSlEWPk-sEJRP3R3x--Ytdx1Fhg'
os.environ["AWS_DEFAULT_REGION"] = 'us-east-1'
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])
#with fs.open("s3://lab/main_df.parquet") as f:
  # df = pd.read_parquet(f)

path="lab/dblp.nt.gz"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 87.0 MB/s  0:00:00

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
authors = {}
publications = {}

with fs.open(path, "rb") as f:
    with gzip.GzipFile(fileobj=f) as gz:
        for i, line in enumerate(gz):
            line = line.decode("utf-8", errors="ignore").strip()
            if not line or line.startswith("#"):
                continue

            # Split the triple
            parts = line.split(" ", 2)
            if len(parts) != 3:
                continue

            subj, pred, obj = parts
            subj = subj.strip("<>")
            pred = pred.strip("<>")
            obj = obj.strip(" .").strip("<>")  # remove surrounding <>, or keep literal quotes

            # Example: store author labels by PID
            if pred.endswith("label") and "/pid/" in subj:
                authors[subj] = obj.strip('"')

            # Example: store publications with DOI
            if pred.endswith("doi"):
                publications[subj] = obj.strip('"')

            # Optional: progress print every million lines
            if i % 1_000_000 == 0:
                print(f"Processed {i} lines...")

print(f"Found {len(authors)} authors and {len(publications)} publications with DOI")
print(list(authors.items())[:5])
print(list(publications.items())[:5])

Processed 0 lines...
Processed 1000000 lines...
Processed 2000000 lines...
Processed 3000000 lines...
Processed 4000000 lines...
Processed 5000000 lines...
Processed 6000000 lines...
Processed 7000000 lines...
Processed 8000000 lines...
Processed 9000000 lines...
Processed 10000000 lines...
Processed 11000000 lines...
Processed 12000000 lines...
Processed 13000000 lines...
Processed 14000000 lines...
Processed 15000000 lines...
Processed 16000000 lines...
Processed 17000000 lines...
Processed 18000000 lines...
Processed 19000000 lines...
Processed 20000000 lines...
Processed 21000000 lines...
Processed 22000000 lines...
Processed 23000000 lines...
Processed 24000000 lines...
Processed 25000000 lines...
Processed 26000000 lines...
Processed 27000000 lines...
Processed 28000000 lines...
Processed 29000000 lines...
Processed 30000000 lines...
Processed 31000000 lines...
Processed 32000000 lines...
Processed 33000000 lines...
Processed 34000000 lines...
Processed 35000000 lines...
Processe

In [12]:
df=pd.DataFrame({"pid":authors.keys(),"author":authors.values()})

In [14]:
df

,pid,author
0,https://dblp.org/pid/52/2402-2,Joaquim Macedo 0002
1,https://dblp.org/pid/52/5816,T. J. Grebinski
2,https://dblp.org/pid/52/3176,Aharon Abadi
3,https://dblp.org/pid/52/1095-2,Muhammad Naeem 0002
4,https://dblp.org/pid/52/11152,Killian Meehan
...,...,...
3930940,https://dblp.org/pid/79/3452,Eric L. Schwartz
3930941,https://dblp.org/pid/79/2667,Qiu Bingyu
3930942,https://dblp.org/pid/79/1529,Sameh Al-Natour
3930943,https://dblp.org/pid/79/8266,Hendrikus Smid


In [13]:
with fs.open("s3://lab/pids.csv", 'wb') as f:
    df.to_csv(f, index=False)

In [ ]:
import requests
import pandas as pd
import time
import os

# Your full authors DataFrame
# df_authors = pd.DataFrame({"pid": [...]})

endpoint = "https://sparql.dblp.org/sparql"
headers = {
    "Accept": "application/sparql-results+json",
    "User-Agent": "Python SPARQL Client/1.0"
}

batch_size = 500_000  # save every 500k rows
all_results = []
file_counter = 1

for i, row in df.iterrows():
    pid = row["pid"]

    query = f"""
    PREFIX dblp: <https://dblp.org/rdf/schema#>
    PREFIX cito: <http://purl.org/spar/cito/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT ?label ?year (COUNT(DISTINCT ?cite) AS ?cites) (?publ AS ?dblp) (SAMPLE(?dois) AS ?doi)
    WHERE {{
      VALUES ?author {{ <{pid}> }}
      ?publ dblp:authoredBy ?author .
      ?publ rdfs:label ?label .
      ?publ dblp:omid ?omids .
      OPTIONAL {{ ?cite cito:hasCitedEntity ?omids . }}
      OPTIONAL {{ ?publ dblp:yearOfPublication ?year . }}
      OPTIONAL {{ ?publ dblp:doi ?dois . }}
    }}
    GROUP BY ?label ?year ?publ
    ORDER BY DESC(?cites)
    LIMIT 100
    """

    try:
        response = requests.get(endpoint, params={"query": query}, headers=headers, timeout=30)
        if response.status_code != 200:
            print(f"Error for PID {pid}: {response.status_code}")
            continue

        data = response.json()

        for result in data["results"]["bindings"]:
            all_results.append({
                "author_pid": pid,
                "label": result.get("label", {}).get("value", ""),
                "year": result.get("year", {}).get("value", ""),
                "dblp_uri": result.get("dblp", {}).get("value", ""),
                "doi": result.get("doi", {}).get("value", ""),
                "citations": int(result.get("cites", {}).get("value", "0"))
            })

    except Exception as e:
        print(f"Exception for PID {pid}: {e}")
        continue

    # Save to CSV every batch_size rows
    if len(all_results) >= batch_size:
        df_batch = pd.DataFrame(all_results)
        filename = f"dblp_citations_batch_{file_counter}.csv"
        df_batch.to_csv(filename, index=False, encoding="utf-8")
        print(f"Saved {len(all_results)} rows to {filename}")
        all_results = []
        file_counter += 1

    # Small delay to avoid overloading endpoint
    time.sleep(0.1)

# Save remaining rows
if all_results:
    df_batch = pd.DataFrame(all_results)
    filename = f"dblp_citations_batch_{file_counter}.csv"
    df_batch.to_csv(filename, index=False, encoding="utf-8")
    print(f"Saved final {len(all_results)} rows to {filename}")


In [3]:
import pandas as pd
f=pd.read_csv("/home/onyxia/work/mem/citation_results/citations_chunk_1.csv")

In [5]:
(f["citation_count"].isna()).sum()

np.int64(500000)